In [80]:
import os
import redis
import hashlib
from metaflow import FlowSpec, step, S3
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

# Ejemplo de predicción de modelo usando Redis

Este notebook muestra cómo obtener predicciones de un modelo que produce predicciones en lotes. Las predicciones se cargaron en Redis. 

En este notebook, recuperamos las predicciones del modelo almacenadas en Redis. Los datos deben ser leídos, convertidos a cadenas y luego hasheados. Con este enfoque, podemos verificar si los datos existen en Redis y recuperar la predicción correspondiente. En caso de que los datos no existan, asignamos un valor de cero. Esta estrategia simula cómo podría comportarse un servicio en producción ante casos no contemplados.

La ventaja de utilizar Redis en este contexto radica en su capacidad para almacenar datos de forma eficiente en memoria, lo que permite un acceso rápido a las predicciones previamente calculadas. Además, Redis ofrece funcionalidades de almacenamiento en caché y persistencia de datos, lo que garantiza la disponibilidad y la integridad de las predicciones incluso en entornos de producción de alta demanda.

In [81]:
# Conectamos al servidor redis (asegúrate de que el docker compose esté corriendo)
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Configuración de las credenciales de acceso a AWS S3 (minio)
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123"
os.environ['AWS_ENDPOINT_URL_S3'] = "http://localhost:9000"

In [82]:
# Obtención de datos para prueba
df = pd.read_csv("./breast_cancer.csv", header=None)

# Sampleamos 100 valores al azar
df_temp = df.sample(50)

test_values = df_temp.values.tolist()

s3 = S3(s3root="s3://amqtp/")
scaler_obj = s3.get("scaler.pkl")
with open(scaler_obj.path, 'rb') as f:
    scaler = pickle.load(f)

# Aplicamos el scaler a los datos
scaled_values = scaler.transform(df_temp)

# Conversión de valores a cadenas y hash
# Esto debería implementarse en el pipeline. Dado que los números de punto flotante pueden 
# presentar problemas debido a pequeñas variaciones, se podría considerar redondearlos.
string_values = [' '.join(map(str, sublist)) for sublist in scaled_values]
hashed_values = [hashlib.sha256(substring.encode()).hexdigest() for substring in string_values]

# Inicializamos un diccionario para almacenar las salidas del modelo
model_outputs = {}

# Obtenemos las predicciones almacenadas en Redis
for hash_key in hashed_values:
    model_outputs[hash_key] = r.hgetall(f"predictions:{hash_key}")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Veamos la salida del modelo para diferentes entradas:

In [83]:
x = []
y = []
target_tree = []
target_svc = []
target_knn = []
target_reglog = []

print("Salidas de los modelos para las primeras 5 entradas:")
for index, test_value in enumerate(test_values):
    hash_key = hashed_values[index]
    tree_prediction = model_outputs[hash_key].get('tree', 'No disponible')
    svc_prediction = model_outputs[hash_key].get('svc', 'No disponible')
    knn_prediction = model_outputs[hash_key].get('knn', 'No disponible')
    reglog_prediction = model_outputs[hash_key].get('reglog', 'No disponible')
    x.append(test_value[0])
    y.append(test_value[6])
    target_tree.append(tree_prediction)
    target_svc.append(svc_prediction)
    target_knn.append(knn_prediction)
    target_reglog.append(reglog_prediction)

    
    print(f"\nPara la entrada: {test_value}")
    print(f"El modelo tree predice: {tree_prediction}")
    print(f"El modelo svc predice: {svc_prediction}")
    print(f"El modelo knn predice: {knn_prediction}")
    print(f"El modelo reglog predice: {reglog_prediction}")
    

print("\nSe han mostrado las predicciones para las primeras 5 entradas.")

Salidas de los modelos para las primeras 5 entradas:

Para la entrada: ['20.55', '20.86', '137.8', '1308.0', '0.1046', '0.1739', '0.2085', '0.1322', '0.2127', '0.06251', '0.6986', '0.9901', '4.706', '87.78', '0.004578', '0.02616', '0.04005', '0.01421', '0.01948', '0.002689', '24.3', '25.48', '160.2', '1809.0', '0.1268', '0.3135', '0.4433', '0.2148', '0.3077', '0.07569']
El modelo tree predice: Maligno
El modelo svc predice: Maligno
El modelo knn predice: Maligno
El modelo reglog predice: Maligno

Para la entrada: ['17.6', '23.33', '119.0', '980.5', '0.09289', '0.2004', '0.2136', '0.1002', '0.1696', '0.07369', '0.9289', '1.465', '5.801', '104.9', '0.006766', '0.07025', '0.06591', '0.02311', '0.01673', '0.0113', '21.57', '28.87', '143.6', '1437.0', '0.1207', '0.4785', '0.5165', '0.1996', '0.2301', '0.1224']
El modelo tree predice: Maligno
El modelo svc predice: Maligno
El modelo knn predice: Maligno
El modelo reglog predice: Maligno

Para la entrada: ['11.13', '22.44', '71.49', '378.4', 